In [1]:
import pandas as pd
import twitter, json,re,sqlalchemy
from slackclient import SlackClient
from sqlalchemy.sql import select
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table,Column,Integer,String,Enum,TEXT,create_engine,UniqueConstraint

### Define database tables

In [2]:
Base = declarative_base()

In [3]:
class Project_status(Base):
  '''
  Table schema for project status
  '''
  __tablename__ = 'project_status'
  __table_args__ = ( UniqueConstraint('project_igf_id'),)

  project_id        = Column(Integer, primary_key=True, nullable=False)
  project_igf_id    = Column(String(50), nullable=False)
  total_sample      = Column(Integer)
  sample_with_fastq = Column(Integer)

class Sample_status(Base):
  '''
  Table schema for sample status
  '''
  __tablename__ = 'sample_status'
  __table_args__ = ( UniqueConstraint('sample_igf_id'),)

  sample_id        = Column(Integer, primary_key=True, nullable=False)
  sample_igf_id    = Column(String(50), nullable=False)
  fastq_read_count = Column(Integer)
    
class Seqrun_status(Base):
  '''
  Table schema for seqrun status
  '''
  __tablename__ = 'seqrun_status'
  __table_args__ = ( UniqueConstraint('seqrun_igf_id'),)

  seqrun_id             = Column(Integer, primary_key=True, nullable=False)
  seqrun_igf_id         = Column(String(50), nullable=False)
  demulieplexing_status = Column(Enum('SEEDED', 'RUNNING', 'FINISHED', 'FAILED', 'UNKNOWN'), nullable=False, server_default='UNKNOWN')

### Create database table using SQlite

In [4]:
engine=create_engine('sqlite:///:memory:')
Base.metadata.create_all(engine)
Session=sessionmaker(bind=engine)
session=Session()

### Define data loading methos for database

In [5]:
def insert_dataframe_row(session, table, df):
    df=df.fillna('')
    df_dict=df.to_dict()
    try:
        df_dict={ key:value for key, value in df_dict.items() if value} # filter any empty value
        obj=table(**df_dict)
        session.add(obj)
        session.flush()
    except Exception as e:
        print("Failed to insert row, error: {}".format(e))
        session.rollback()

### Load demo data to db

In [6]:
project_data=pd.DataFrame([{'project_igf_id':'IGFP001','total_sample':30,'sample_with_fastq':25}])
sample_data=pd.DataFrame([{'sample_igf_id':'IGFS001','fastq_read_count':3000000}])
seqrun_data=pd.DataFrame([{'seqrun_igf_id':'171003_M03291_0089_000000000-TEST','demulieplexing_status':'RUNNING'}])
project_data.apply(lambda x: insert_dataframe_row(session,Project_status, x), axis=1);
sample_data.apply(lambda x: insert_dataframe_row(session,Sample_status, x), axis=1);
seqrun_data.apply(lambda x: insert_dataframe_row(session,Seqrun_status, x), axis=1);
session.commit()

### Check demo data

In [7]:
p_query=session.query(Project_status)
pd.read_sql(p_query.statement, session.bind)

,project_id,project_igf_id,total_sample,sample_with_fastq
0,1,IGFP001,30,25


In [8]:
sa_query=session.query(Sample_status)
pd.read_sql(sa_query.statement, session.bind)

,sample_id,sample_igf_id,fastq_read_count
0,1,IGFS001,3000000


In [9]:
sr_query=session.query(Seqrun_status)
pd.read_sql(sr_query.statement, session.bind)

,seqrun_id,seqrun_igf_id,demulieplexing_status
0,1,171003_M03291_0089_000000000-TEST,RUNNING


### Define da data fetch methods

In [10]:
def get_project_status(project_igf_id,session):
    '''
    Function for fetching project status
    '''
    try:
        p_query=session.\
                query(Project_status).\
                filter(Project_status.project_igf_id==project_igf_id)
        project_status=p_query.one_or_none()
        return project_status
    except Exception as e:
        print(e)

In [11]:
def get_sample_status(sample_igf_id,session):
    '''
    Function for fetching sample status
    '''
    try:
        sa_query=session.\
                 query(Sample_status).\
                 filter(Sample_status.sample_igf_id==sample_igf_id)
        sample_status=sa_query.one_or_none()
        return sample_status
    except Exception as e:
        print(e)

In [12]:
def get_seqrun_status(seqrun_igf_id,session):
    '''
    Function for fetching seqrun status
    '''
    try:
        sr_query=session.\
                 query(Seqrun_status).\
                 filter(Seqrun_status.seqrun_igf_id==seqrun_igf_id)
        seqrun_status=sr_query.one_or_none()
        return seqrun_status
    except Exception as e:
        print(e)

### Method for generating message for Slack

In [13]:
def calculate_reply(user_input,session):
    '''
    Function for calculation reply for the user input
    '''
    try:
        pattern=re.compile(r'^<@\w+>(\s+)?(\S+)(\s+)?:(\s+)?(\S+)$')
        m = re.search(pattern,user_input)
        if m:
            key=m.group(2)
            value=m.group(5)
            if key.lower()=='project':
                project_data=get_project_status(project_igf_id=value,
                                                session=session)
                if project_data:
                    msg='project {0} has {1} samples, currently we have {2} samples with fastq'.\
                        format(value,\
                               project_data.total_sample, \
                               project_data.sample_with_fastq)
                else:
                    msg='no data found for project {0}'.format(value)
            elif key.lower()=='sample':
                sample_data=get_sample_status(sample_igf_id=value,
                                              session=session)
                if sample_data:
                    msg='sample {0} has total {1} fastq reads'.\
                        format(value,\
                               sample_data.fastq_read_count)
                else:
                    msg='no data found for sample {0}'.format(value)
            elif key.lower()=='seqrun':
                seqrun_data=get_seqrun_status(seqrun_igf_id=value,
                                              session=session)
                if seqrun_data:
                    msg='sequencing run {0} has following status for demultiplexing pipeline: {1}'.\
                        format(value,\
                               seqrun_data.demulieplexing_status)
                else:
                    msg='no data found for sequencing run {0}'.format(value)
            else:
                msg='No option present for keyword {0}, available options are: project, sample, seqrun'.\
                     format(key)
        else:
            msg='No match found'
        return msg
    except Exception as e:
        print(e)

### Test reply with demo data

In [14]:
calculate_reply(user_input='<@UAAA42>  project: IGFP001',session=session)

'project IGFP001 has 30 samples, currently we have 25 samples with fastq'

In [15]:
calculate_reply(user_input='<@UAAA42>  sample: IGFS001',session=session)

'sample IGFS001 has total 3000000 fastq reads'

In [16]:
calculate_reply(user_input='<@UAAA42>  seqrun: 171003_M03291_0089_000000000-TEST',session=session)

'sequencing run 171003_M03291_0089_000000000-TEST has following status for demultiplexing pipeline: RUNNING'

In [17]:
calculate_reply(user_input='<@UAAA42>  sample1: IGFS001',session=session)

'No option present for keyword sample1, available options are: project, sample, seqrun'

### Load Slack config

In [18]:
with open('slack_conf.json','r') as json_data:
    slack_token=json.load(json_data)

In [19]:
sc = SlackClient(slack_token['slack_token'])
slack_token['slack_token']=''
slack_bot_id=slack_token['slack_token']
slack_channel=slack_token['slack_channel']

### Define method for parsing user input and sending reply

In [20]:
def post_slack_message(message, channel_id, user_id, attachments=None, thread_ts=None):
    '''
    message: slack messsage
    channel_id: slack channel id
    user_id: slack bot user id
    thread_ts: slack thread id, if available
    '''
    sc.api_call(
   "chat.postMessage",
   channel=channel_id,
   user=user_id,
   thread_ts=thread_ts,    
   is_im=True, 
   attachments=attachments,
   text=message
  )



In [21]:
def parse_slack_output(slack_rtm_output, bot_id, channel_id):
    '''
    slack_rtm_output: slack realtime stream
    bot_id: slack bot id
    channel_id: slack channel id
    '''
    if isinstance(slack_rtm_output, list) and \
       len(slack_rtm_output) > 0:
            for output in slack_rtm_output:
                if output and \
                   'text' in output and \
                   '@'+bot_id in output['text'] and \
                   channel_id in output['channel']:
                        yield output


### Slack RTM process for bot

In [22]:
try:
    if sc.rtm_connect():
        while True:
            for output in parse_slack_output(slack_rtm_output=sc.rtm_read(),\
                                             bot_id=slack_bot_id, \
                                             channel_id=slack_channel):
                message=calculate_reply(user_input=output['text'],session=session)
                post_slack_message(message=message, \
                                   channel_id=slack_channel, \
                                   user_id=slack_bot_id, \
                                   thread_ts=output['ts'])
except KeyboardInterrupt:
    print('Stopped bot')
except:
    raise

Stopped bot
